In [108]:
import os
import importlib
import pretty_midi
import fluidsynth
import midi_tm
import music21
from IPython.display import Audio

In [4]:
path = 'C:\\code\\python_dev\\rl_gym\\midi'

In [104]:
importlib.reload(midi_tm)

<module 'midi_tm' from 'c:\\code\\python_dev\\IMMNE\\midi_tm.py'>

In [12]:
files_list = midi_tm.find_midis(path, max=10)

Found 10 .mid files (Max limit reached)


In [98]:
def midi_statistics(files_list):

    song_names = []
    parsed_midis = []
    instruments_table = []
    instrument_names = []

    for file in files_list:
        try:
            pm = pretty_midi.PrettyMIDI(file)
        except:
            continue # skip bad files
        
        song_names.append(os.path.split(file)[1])
        parsed_midis.append(pm)

        for instrument in pm.instruments:
            if instrument.is_drum: 
                continue # skip the drums
            
            instrument.remove_invalid_notes()
            if len(instrument.notes) >= 42: # a most arbitrary number
                if instrument.name not in instrument_names:
                    instrument_names.append(instrument.name)
                    instruments_table.append([instrument.name, 0, pretty_midi.program_to_instrument_class(instrument.program)])
                                    
                instruments_table[instrument_names.index(instrument.name)][1] += len(instrument.notes)
    
    print(f'Analyzed {len(song_names)} of {len(files_list)} files.')

    return song_names, parsed_midis, instruments_table

In [89]:
a = [[1, 2, 3], [1, 2, 3]]
a[0][1]

2

In [99]:
song_names, instruments = midi_statistics(files_list)


Analyzed 9 of 10 files.


In [138]:
pm = pretty_midi.PrettyMIDI(files_list[0])
print('There are {} time signature changes'.format(len(pm.time_signature_changes)))
print('There are {} instruments:'.format(len(pm.instruments)))

instrument_notes = []

for instrument in pm.instruments:

    note_times = []
    chords = 0
    notes = []
    notes_in_this_chord = 0
    notes_in_chords = 0

    for c, note in enumerate(instrument.notes[:-1]):
        if instrument.notes[c].start == instrument.notes[c+1].start:
            # part of chord
            notes_in_this_chord += 1
            notes_in_chords += 1

            if instrument.notes[c].start not in note_times:
                # chord not encountered before
                chords += 1
                note_times.append(note.start)
        else:
            notes.append(instrument.notes[c-notes_in_this_chord:c+1])
            notes_in_this_chord = 0

    notes_in_chords += chords #counting the first note twice equals counting last note

    print(f'Instrument {instrument.name} has: \
    {len(instrument.notes)-notes_in_chords} notes, {chords} chords, {len(instrument.pitch_bends)} pitch_bends {len(instrument.control_changes)} control_changes')
    
    instrument_notes.append(notes)

#    print(f'Found {chords} chords containing {notes_in_chords} notes')

            

There are 1 time signature changes
There are 5 instruments:
Instrument Vocal has:     369 notes, 0 chords, 1 pitch_bends 128 control_changes
Instrument ElecGtr has:     25 notes, 152 chords, 1 pitch_bends 110 control_changes
Instrument Bass has:     47 notes, 571 chords, 1 pitch_bends 107 control_changes
Instrument SteelGtr has:     47 notes, 571 chords, 1 pitch_bends 120 control_changes
Instrument Rhythm&SE has:     365 notes, 380 chords, 1 pitch_bends 102 control_changes


In [152]:
print(f'my extract: {instrument_notes[1][1]}')
for chord in instrument_notes[1]:
    if len(chord) > 2:
        notes = []
        print(f'{chord}')
        for note in chord:
            notes.append(pretty_midi.note_number_to_name(note.pitch))
        try:
            print(notes)
            print(music21.chord.Chord(notes).root())
        except:
            print(f'Could not deduct root')
#print(f'actual: {pm.instruments[1].notes[0:5]}')

my extract: [Note(start=2.250000, end=2.416667, pitch=64, velocity=79), Note(start=2.250000, end=2.416667, pitch=59, velocity=79)]
[Note(start=11.750000, end=12.666667, pitch=61, velocity=79), Note(start=11.750000, end=12.666667, pitch=64, velocity=79), Note(start=11.750000, end=12.666667, pitch=68, velocity=79)]
['C#4', 'E4', 'G#4']
C#4
[Note(start=12.750000, end=13.666667, pitch=71, velocity=79), Note(start=12.750000, end=13.666667, pitch=63, velocity=79), Note(start=12.750000, end=13.666667, pitch=66, velocity=79)]
['B4', 'D#4', 'F#4']
B4
[Note(start=13.750000, end=14.833333, pitch=64, velocity=79), Note(start=13.750000, end=14.833333, pitch=61, velocity=79), Note(start=13.750000, end=14.833333, pitch=69, velocity=79)]
['E4', 'C#4', 'A4']
A4
[Note(start=15.000000, end=15.333333, pitch=68, velocity=79), Note(start=15.000000, end=15.333333, pitch=61, velocity=79), Note(start=15.000000, end=15.333333, pitch=64, velocity=79)]
['G#4', 'C#4', 'E4']
C#4
[Note(start=15.500000, end=17.322917

In [50]:
some_music = pretty_midi.PrettyMIDI()
some_music.instruments.append(pm.instruments[1])
some_music.instruments.append(pm.instruments[2])
some_music.write('BassElecGtr.mid')

In [101]:
for instrument in pm.instruments:

pretty_midi.note_number_to_name(pm.instruments[0].notes[0].pitch)

'G#4'

In [76]:
t_mat = pm.get_pitch_class_transition_matrix(normalize=True, time_thresh=30.0)

In [106]:
midis = midi_tm.parse_midi_files(midi_tm.find_midis(path, max=10), max=10)

Found 10 .mid files (Max limit reached)
Parsing 10 files0.


In [109]:
s2 = music21.instrument.partitionByInstrument(midis[0])

In [113]:
s2.parts[0].notes

<music21.stream.iterator.StreamIterator for Part:Brass @:0>

In [92]:
long_list = midi_tm.get_notes_list(midis)

In [87]:
long_list = midi_tm.notes_cleaning(long_list)

In [88]:
t_matrix, ui, ui_freq = midi_tm.transition_matrix(long_list)

Traversing list of length 140479...
Done. Found 73 unique items.


In [89]:
long_list_12 = midi_tm.notes_compress12(long_list)

In [90]:
t_matrix_12, ui_12, ui_freq_12 = midi_tm.transition_matrix(long_list_12)

Traversing list of length 140479...
Done. Found 10 unique items.


In [91]:
importlib.reload(plotting)

fig = plotting.fig_tm(t_matrix_12, ui_12, ui_freq_12)

fig.show()

In [93]:
fig2 = plotting.fig_tm(t_matrix, ui, ui_freq)

fig2.show()

In [84]:
importlib.reload(midi_tm)

<module 'midi_tm' from 'c:\\code\\python_dev\\IMMNE\\midi_tm.py'>

In [114]:
def list_instruments(midi):
    partStream = midi.parts.stream()
    print("List of instruments found on MIDI file:")
    for p in partStream:
        aux = p
        print (p.partName)

In [133]:
sp = midi.realtime.StreamPlayer(midis[22])
midis[22].write('midi', 'c:\\code\\python_dev\\IMMNE\\test.mid')
from music21 import converter, instrument, midi, note, chord, pitch, corpus

mf = midi.MidiFile()
mf.open(files_list[22])
mf.read()
mf.close()

base_midi = midi.translate.midiFileToStream(mf)


In [123]:
print(len(midis[0].flatten().getElementsByClass(music21.note.Note)))
print(f'433+89+47={433+89+47}')

569
433+89+47=569


In [118]:
list_instruments(midis[0])
s2 = music21.instrument.partitionByInstrument(midis[0])
print(f'#flat notes: {len(midis[0].flat.notes)}')
n = 0

stuff_to_parse = []
for c, part in enumerate(s2.parts):
    notes = 0
    controls = 0
    chords = 0

    stuff_to_parse.append(s2.parts[c].recurse())
    for element in stuff_to_parse:
        if isinstance(element, music21.note.Note):
            notes += 1
        elif isinstance(element, music21.chord.Chord):
            chords += 1
        

    print(f'part {c} has {len(part)} elements: {len(part.notes)')

print(f'{len(part')

notes = []
flat_notes = []
chords = []

for instrument in notes_to_parse:
    # should add: if instrument.name is in [Strings, Guitars] or similar
    notes_row = []
    flat_notes_row = []
    chords_row = []
    
    for element in instrument:
        if isinstance(element, music21.note.Note):
            notes_row.append(element)
            flat_notes_row.append(str(element.pitch))
    
        elif isinstance(element, music21.chord.Chord):
            chords_row.append(element)
            #notes.append('.'.join(str(n) for n in element.normalOrder)) not extracting notes

    notes.append(notes_row)
    flat_notes.append(flat_notes_row)
    chords.append(chords_row)

for c, part in enumerate(notes):
    print(f'instrument (part) {c} has {len(part)} notes')

for c, part in enumerate(chords):
    print(f'instrument (part) {c} has {len(part)} chords')


List of instruments found on MIDI file:
Vocal
ElecGtr
Bass
SteelGtr
Rhythm&SE
#flat notes: 2672
part 0 has 795 elements
part 1 has 1666 elements
part 2 has 5 elements
part 3 has 1249 elements
part 4 has 1440 elements
instrument (part) 0 has 433 notes
instrument (part) 1 has 89 notes
instrument (part) 2 has 0 notes
instrument (part) 3 has 47 notes
instrument (part) 4 has 0 notes
instrument (part) 0 has 0 chords
instrument (part) 1 has 779 chords
instrument (part) 2 has 0 chords
instrument (part) 3 has 574 chords
instrument (part) 4 has 0 chords


In [137]:
sp.play()

KeyboardInterrupt: 

In [138]:
sp.stop()

In [ ]:
# folder path
dir_path = 'C://code/python_dev/rl_gym/midi/'
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

In [6]:
midi_path = 'C://code/python_dev/rl_gym/midi/'
artists = os.listdir(root_path)
songlengths = [0]*len(artists)

for i, artist in enumerate(artists):
    songlengths[i] += len(os.listdir(midi_path+artist))

zipped = zip(songlengths, artists)
sorted_list = sorted(zipped, key = lambda x: x[0])

In [5]:
import pretty_midi
import midi_tm

In [13]:
files_list = midi_tm.find_midis(midi_path, max=800, artist='Bach')
song_names, parsed_midis = midi_tm.parse_to_midi(files_list)
instruments_table = midi_tm.midi_statistics(parsed_midis)

Found 100 .mid files
Parsed 100 of 100 files.
Analyzed 100 midis, with 81 unique instruments
